In [2]:
import pandas as pd

### extraction Data 

In [3]:
df = pd.read_csv("Traces.csv", header=None)  

# Créer les noms de colonnes pour le temps
time_cols = [str(i+1) for i in range(10000)]
df.columns = time_cols

#générer les noms des lignes (cordonnées des satellites):
coord_names = []
for i in range(1, 101):  # 100 satellites
    coord_names.append(f"satx{i}")
    coord_names.append(f"saty{i}")
    coord_names.append(f"satz{i}")
df["coords"] = coord_names
df = df.set_index("coords")

dft = df.transpose()  # maintenant chaque ligne = un instant

### convert to swarm

In [ ]:
connection_range = 60
portee1 =20
portee2=40
portee3= 60
nanosat_count = 100
max_temps = 100  # pour test rapide

for t in range(len(dft)):
    coords = dft.iloc[t]
    nodes = []
    
    # Créer les objets Node
    for i in range(1, nanosat_count + 1):
        x = coords[f'satx{i}']
        y = coords[f'saty{i}']
        z = coords[f'satz{i}']
        nodes.append(Node(id=i, x=x, y=y, z=z))

    # Créer un graphe pondéré
    G = nx.Graph()
    for node in nodes:
        G.add_node(node.id)

    # Ajouter les arêtes avec poids selon la distance
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            dist = nodes[i].compute_dist(nodes[j])
            if dist <= portee1:
                weight = 1
            elif dist <= portee2:
                weight = 2
            elif dist <= portee3:
                weight = 3
            else:
                continue  
            G.add_edge(nodes[i].id, nodes[j].id, weight=weight)
        # ----- Étape 1 : Métriques de base (graphe non divisé) -----
    metriques_avant = {
        "time": t,
        "algo": "none",
        "attack": "none",
        "efficacite": network_efficiency(G),
        "cout": routing_cost(G),
        "robustesse": flow_robustness(G),
        "redondance": path_redundancy(G),
        "disparite": path_disparity(G),
        "criticite": critical_nodes(G)
    }
    resultats.append(metriques_avant)

    # ----- Étape 2 : Pour chaque algorithme de division -----
        groupes = swarm.MDRW(n=5)

        # Construire group_dict
        group_dict = {}
        for group_id, sub_swarm in groupes.items():
            for n in sub_swarm.nodes:
                group_dict[n.id] = group_id

        # ----- Métriques post-division -----
        metriques_div = {
            "time": t,
            "algo": algo,
            "attack": "none",
            "efficacite": network_efficiency(G, divided=True, group_dict=group_dict),
            "cout": routing_cost(G, divided=True, group_dict=group_dict),
            "robustesse": flow_robustness(G, divided=True, group_dict=group_dict),
            "redondance": path_redundancy(G, divided=True, group_dict=group_dict),
            "disparite": path_disparity(G, divided=True, group_dict=group_dict),
            "criticite": critical_nodes(G, divided=True, group_dict=group_dict)
        }
        resultats.append(metriques_div)

        # ----- Étape 3 : Appliquer les attaques -----
          attack_type= "random"
           nb_noeuds_supprimes = 40 
            G_copy = G.copy()
            BC = node_criticality(G_copy if attack_type == "cible" else None,
                                  divided=True, group_dict=group_dict)
            if attack_type == "random":
                nodes_to_remove = random.sample(list(G_copy.nodes), nb_noeuds_supprimes)
            else:  # "cible"
                nodes_to_remove = sorted(BC, key=BC.get, reverse=True)[:nb_noeuds_supprimes]

            G_copy.remove_nodes_from(nodes_to_remove)

            # ----- Métriques post-attaque -----
            metriques_attaque = {
                "time": t,
                "algo": algo,
                "attack": attack_type,
                "efficacite": network_efficiency(G_copy, divided=True, group_dict=group_dict),
                "cout": routing_cost(G_copy, divided=True, group_dict=group_dict),
                "robustesse": flow_robustness(G_copy, divided=True, group_dict=group_dict),
                "redondance": path_redundancy(G_copy, divided=True, group_dict=group_dict),
                "disparite": path_disparity(G_copy, divided=True, group_dict=group_dict),
                "criticite": critical_nodes(G_copy, divided=True, group_dict=group_dict)
            }
            resultats.append(metriques_attaque)

In [ ]:
# ========== Sauvegarde en CSV ==========
df_res = pd.DataFrame(resultats)
df_res.to_csv("resultats_swarm_simulation.csv", index=False)